In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from pyswarm import pso
import pandas as pd

class PSOOptimizer(optimizers.Optimizer):
    def __init__(self, num_particles=10, c1=0.5, c2=0.5, w=0.5, max_iterations=100, name='PSOOptimizer', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_particles = num_particles
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.max_iterations = max_iterations

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_particles': self.num_particles,
            'c1': self.c1,
            'c2': self.c2,
            'w': self.w,
            'max_iterations': self.max_iterations
        })
        return config

    def get_gradients(self, loss, params):
        gradients = []
        lb = [-1.0] * len(params)  
        ub = [1.0] * len(params) 
        particles = [tf.Variable(p) for p in pso(self.loss, lb, ub, 
                                                  swarmsize=self.num_particles, maxiter=self.max_iterations, 
                                                  debug=False, omega=self.w, phip=self.c1, phig=self.c2)[0]]

        for p, new_p in zip(params, particles):
            gradients.append(new_p - p)

        return gradients

    def loss(self, params):
        self.model.set_weights(params)

        x_batch, y_batch = self.model._standardize_user_data(self.inputs, self.targets)
        return self.model.total_loss(x_batch, y_batch, sample_weight=self.model.sample_weights)

    def _resource_apply_dense(self, grad, var):
        updated_var = var - grad
        return var.assign(updated_var)

    def _resource_apply_sparse(self, grad, var, indices):
        raise NotImplementedError

In [32]:
model = models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(13,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer=PSOOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
df = pd.read_csv(r"C:\Users\91755\Downloads\Bank_Personal_Loan_Modelling.csv")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [34]:
from sklearn.model_selection import train_test_split

x = df.loc[:, df.columns != 'Personal Loan'].values
y = df.loc[:,df.columns == 'Personal Loan'].values


In [ ]:
model.fit(x, y, epochs=10, batch_size=32, validation_data=(x, y))

In [37]:
# Optimized weights
model.get_weights()

[array([[ 3.72690558e-02,  2.72956997e-01, -8.53309333e-02,
         -1.80405080e-02,  1.52795672e-01,  1.33176595e-01,
         -1.32529154e-01, -2.41034582e-01,  2.63456851e-01,
         -2.55953938e-01, -7.76914656e-02,  7.07767606e-02,
         -2.30142593e-01, -1.59282625e-01, -7.82716125e-02,
          2.53901213e-01,  2.44964033e-01,  2.82943249e-02,
          4.68655229e-02, -2.40440533e-01,  1.09129518e-01,
          3.98105979e-02,  4.21965420e-02, -1.30617872e-01,
         -6.89809620e-02,  1.77546740e-01, -1.10752612e-01,
          1.80458456e-01,  1.86062515e-01,  1.74614549e-01,
          2.57343501e-01, -8.39130729e-02,  2.62033194e-01,
         -7.77050406e-02, -1.07380942e-01,  2.09360272e-01,
          2.04017103e-01, -2.15788752e-01, -2.71130919e-01,
          1.19946897e-01, -1.73330128e-01, -1.63407147e-01,
         -1.37759581e-01,  2.14718580e-02,  1.59433573e-01,
         -8.83845389e-02, -2.57423580e-01, -9.85562354e-02,
         -8.96271616e-02, -1.50176585e-0